In [1]:
import cv2 
import numpy as np

video_path = "../Train/anomaly/3.mp4"

In [53]:
def extract_lighting_change(frame1, frame2):
    ksize = (10, 10) 
    f1_blur = cv2.blur(frame1, ksize)  
    f2_blur=cv2.blur(frame2,ksize)
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray1, gray2)
    return np.mean(diff)

def extract_blur(frame):
    #фильтр обнаружения краев, если дисперсии мало - есть размытие
    blur = cv2.Laplacian(frame, cv2.CV_64F).var()
    return blur

def extract_camera_angle_change(frame1, frame2):
    sift = cv2.SIFT_create()
    ksize = (10, 10) 
    f1_blur = cv2.blur(frame1, ksize)  
    f2_blur=cv2.blur(frame2,ksize)
    # SIFT определяет ключевые точки и их дескрипторы
    kp1, des1 = sift.detectAndCompute(f1_blur, None)
    kp2, des2 = sift.detectAndCompute(f2_blur, None)

    # проходимся knn матчером по соответствующим точкам обоих кадров
    # если "хороших" совпадений мало - камеру сдвинули
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    return len(good_matches)

def extract_features_from_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    prev_frame = None
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("End of video")
            break
        
        if prev_frame is not None:
            lighting_change = extract_lighting_change(prev_frame, frame)
            blur = extract_blur(frame)
            camera_angle_change = extract_camera_angle_change(prev_frame, frame)
            cv2.setWindowTitle("frame",f"lg:{round(lighting_change,2)} blur:{blur} cac:{camera_angle_change}")
            
        
        prev_frame = frame
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
extract_features_from_video(video_path)

In [29]:
import pandas as pd
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    prev_frame = None
    features_array=[]
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("End of video")
            break
        
        if prev_frame is not None:
            lighting_change = extract_lighting_change(prev_frame, frame)
            blur = extract_blur(frame)
            camera_angle_change = extract_camera_angle_change(prev_frame, frame)
            
            # print("Lighting Change:", lighting_change)
            # print("Blur:", blur)
            # print("Camera Angle Change:", camera_angle_change)
            features_array.append([blur,lighting_change,camera_angle_change])
            
        
        prev_frame = frame
    return pd.DataFrame(data=features_array,columns=["Blur","Lighting Change","Camera Angle Change"])

In [10]:
df=preprocess_video(video_path)

In [11]:
df

,Blur,Lighting Change,Camera Angle Change
0,323.95767,0.221717,1900


In [22]:
import glob
train_path="../Train/*/*.mp4"
files=glob.glob(train_path)
files

['../Train\\anomaly\\0.mp4',
 '../Train\\anomaly\\2.mp4',
 '../Train\\anomaly\\3.mp4',
 '../Train\\anomaly\\4.mp4',
 '../Train\\anomaly\\5.mp4',
 '../Train\\not_anomaly\\0.mp4',
 '../Train\\not_anomaly\\1.mp4',
 '../Train\\not_anomaly\\2.mp4',
 '../Train\\not_anomaly\\3.mp4',
 '../Train\\not_anomaly\\4.mp4',
 '../Train\\not_anomaly\\5.mp4',
 '../Train\\not_anomaly\\6.mp4',
 '../Train\\not_anomaly\\7.mp4',
 '../Train\\not_anomaly\\8.mp4']

In [28]:
for pth in files:
    df=preprocess_video(pth)
    tag=pth.split("\\")[-2]
    name=pth.split("\\")[-1]
    print(name,tag)
    df.to_csv(f"../csv/{tag}/{name[0]}.csv")

0.mp4 anomaly
2.mp4 anomaly
3.mp4 anomaly
4.mp4 anomaly
5.mp4 anomaly
0.mp4 not_anomaly
1.mp4 not_anomaly
2.mp4 not_anomaly
3.mp4 not_anomaly
4.mp4 not_anomaly
5.mp4 not_anomaly
6.mp4 not_anomaly
7.mp4 not_anomaly
8.mp4 not_anomaly
